https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt

# Preparation

In [1]:
from google.colab import drive
drive.mount('/Til')

Mounted at /Til


In [2]:
!pip install transformers sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.7 MB/s eta 0:00:00


In [3]:
!wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto

--2023-02-28 10:16:44--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13451 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

sentencepiece_model 100%[===================>]  13.14K  --.-KB/s    in 0s      

2023-02-28 10:16:44 (62.5 MB/s) - ‘sentencepiece_model.proto’ saved [13451/13451]



In [4]:
! protoc --python_out=. sentencepiece_model.proto

# Counting words

In [5]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [6]:
print(tokenizer.vocab_size)

250054


In [7]:
import pandas as pd
import numpy as np

In [8]:
all_sentences = pd.read_csv('/Til/MyDrive/Colab Notebooks/Til/Data/Bary_bir_to_model.csv')
all_sentences = np.array(all_sentences.krc)

In [9]:
all_sentences[:10]

array(['Теnиз тузлу некди?',
       'Бир jашчыq анасы бла тиnизге jуwунурgа барgанды. Jашчыq qуwаныb, суwgа кёмюлсе, аwузуна суw киргенди, кеси wa тузлу суw. Jунчуgан да этиb, бир уртлам суwну да jутуb, ол jаgаgа чыqgанды. Суwну алай тузлу нек болgанын аnыламай:',
       '- Некди теnиз былай тузлу? – деb сейирсинnенди.',
       'Анасы jашына теnизни хапарын айтханды:',
       '- Эртде-эртде бир бай qарнаш бла бир jарлы qарнаш jашаgандыла. Jарлы qарнаш байgа «менnе муну бла бир тюйчюк бер» деb, табаqны алыb, тилей барgанды. Бай qарнаш бермегенди.',
       'Jарлы, аны юйюнден чыgыb, мыдах болуb, келе турgанлай, qолунда гырjынчыgы бла бир киши чыqgанды аллына. ',
       '- Не айланаса? – деb сорgан кишичик.',
       'Jаш аnа, былай да былай деb, jарлылыq хапарын айтханды.',
       '- Ма, бу гырjынчыqны ал да, таw артына бар. Анда гитче адамчыqла jашайдыла. Башха затха унамай, гырjынчыgыnы тирменчикге аwушдур. Андан арысын кёре барырса, - деb кишичик гырjынчыgын да jашха тутдуруb, jолуна к

In [10]:
from collections import Counter, defaultdict
from tqdm.auto import tqdm, trange
import random
import re

In [11]:
text = 'Моё уважение!!! :) :-\ :-D'

In [12]:
tokenizer.tokenize(text)

['▁Мо', 'ё', '▁уважение', '!!!', '▁:)', '▁:-', '\\', '▁:-', 'D']

In [13]:
tokenizer.prepare_for_tokenization(text)

('Моё уважение!!! :) :-\\ :-D', {})

In [14]:
tokenizer.tokens_trie.split(text)

['Моё уважение!!! :) :-\\ :-D']

In [15]:
tokenizer.sp_model.encode(text)

[7330, 7784, 164999, 1563, 1093, 15771, 41871, 15771, 396]

In [16]:
tokens = tokenizer.tokenize(text)
tokens

['▁Мо', 'ё', '▁уважение', '!!!', '▁:)', '▁:-', '\\', '▁:-', 'D']

In [17]:
char_count = Counter()
for text in tqdm(all_sentences):
    char_count.update(text)

  0%|          | 0/27235 [00:00<?, ?it/s]

In [18]:
PUNCT = '.,-—:)(»«!?–/;„"“…*́№Ёҥ[]”^%+І=і•_􏰀²|}{#‘■>⁠’á<°\§\''
SPACE = '▁'

In [19]:
for k, v in char_count.most_common(200):
    if not re.match('[а-яА-Яa-zA-Z0-9ё\']', k):
        print(k, end='')

 ,.–:!-?«»Ё()…’;ý/"“¬қ”—|‘`

In [20]:
toks = tokenizer.tokenize(text)
toks

['▁G', 'а', 'рш']

In [21]:
def get_words(text, tokenizer, verbose=False):
    toks = tokenizer.tokenize(text)
    words = []
    word = []
    prev_punct = False
    for tok in toks:
        is_punct = tok.lstrip(SPACE) and all(c in PUNCT for c in tok.lstrip(SPACE))
        if tok.startswith(SPACE) or prev_punct != is_punct:
            if word:
                words.append(word)
            word = []
        word.append(tok)
        prev_punct = is_punct
    if word:
        words.append(word)
    if verbose:
        print(words)
    res = words
    # assert tokenizer.decode([tok for t in res for tok in t]) == text
    return res

In [22]:
word_count = Counter()
word2toks = {}
for text in tqdm(all_sentences):
    for word_toks in get_words(text, tokenizer):
        word = ''.join(word_toks)
        word_count[word] += 1
        word2toks[word] = word_toks
print(len(word_count))

  0%|          | 0/27235 [00:00<?, ?it/s]

42463


In [23]:
from copy import deepcopy
word_count2 = deepcopy(word_count)
word2toks2 = deepcopy(word2toks)

# Computing splits

In [24]:
word_count = deepcopy(word_count2)
word2toks = deepcopy(word2toks2)

In [25]:
for k, v in word_count.most_common(30):
    if len(word2toks[k]) > 1:
        print(word2toks[k])

['▁де', 'b']
['▁эд', 'и']
['▁а', 'лай']
['▁у', 'ой']
['▁дей', 'ди']
['▁болу', 'b']
['▁ойр', 'а']


In [26]:
steps = 100_000
min_count = 30  
# default:   0 new tokens, 30 lenght, 0% new tokens
# 100 mindf: 6.6k new tokens, 22 length, 47% new tokens (of sentence length)
# 30 mindf:  20k new tokens, 20 length, 58% new tokens
# 10 mindf: 50K new tokens, 18.5 length, 64% new tokens
extra_vocab = []
extra_counts = []
extra_pairs = []

In [27]:
# word2toks = {''.join(tokenizer.convert_ids_to_tokens(list(w))): w for w in tqdm(word_count)}

In [27]:
pairs_count = Counter()
pair2word = defaultdict(set)
for w, c in tqdm(word_count.items(), total=len(word_count)):
    enc = word2toks[w]
    for pair in zip(enc[:-1], enc[1:]):
        pairs_count[pair] += c
        pair2word[pair].add(w)

  0%|          | 0/42463 [00:00<?, ?it/s]

In [28]:
from heapdict import heapdict
hd = heapdict()
for w, c in pairs_count.items():
    hd[w] = -c

In [29]:
def replace_pair(old_tokens, pair, new_token):
    result = []
    prev = old_tokens[0]
    for tok in old_tokens[1:]:
        if (prev, tok) == pair:
            result.append(new_token)
            prev = None
        else:
            if prev is not None:
                result.append(prev)
            prev = tok
    if prev is not None:
        result.append(prev)
    return result

In [30]:
# rewrite with heapdict
for _ in trange(steps):
    #pair, c = pairs_count.most_common(1)[0]  # это самая времязатратная операция
    pair, c = hd.peekitem()
    c = -c
    
    if c < min_count:
        break
    new_token = ''.join(pair) # instead of BERT-like pair[0] + pair[1][2:]
    extra_vocab.append(pair)
    extra_counts.append(c)
    extra_pairs.append(pair)
    
    # update the vocabulary
    #new_id = len(id2ids)
    #tok2id[new_token] = new_id
    #id2ids.append(id2ids[tok2id[pair[0]]] + id2ids[tok2id[pair[1]]]) 
    
    # calculate the delta for the heap
    delta = Counter()
    for word in list(pair2word[pair]):
        # calculate old and new ways to tokenize the word
        old_toks = word2toks[word]
        # new_toks = " ".join(old_toks).replace(' '.join(pair), new_token).split(" ")
        new_toks = replace_pair(old_toks, pair, new_token)
        word2toks[word] = new_toks
        wc = word_count[word]
        # update the index concerning the tokens of the word
        for old_pair in zip(old_toks[:-1], old_toks[1:]):
            #pairs_count[old_pair] -= wc
            delta[old_pair] -= wc
            if word in pair2word[old_pair]:
                pair2word[old_pair].remove(word)
        for new_pair in zip(new_toks[:-1], new_toks[1:]):
            # pairs_count[new_pair] += wc
            delta[new_pair] += wc
            pair2word[new_pair].add(word)
    # update the heap
    for a_pair, a_delta in delta.items():
        if a_delta == 0:
            continue
        if a_pair not in hd:
            hd[a_pair] = 0
        hd[a_pair] -= a_delta

  0%|          | 0/100000 [00:00<?, ?it/s]

In [31]:
len(extra_pairs)

2237

In [32]:
extra_pairs[:10]

[('g', 'а'),
 ('g', 'ан'),
 ('▁j', 'а'),
 ('▁де', 'b'),
 ('▁эд', 'и'),
 ('▁q', 'а'),
 ('▁к', 'ёр'),
 ('ю', 'й'),
 ('▁q', 'ара'),
 ('n', 'а')]

In [33]:
extra_pairs[-20:]

[('ге', 'ним'),
 ('▁баш', 'лары'),
 ('▁бай', 'лаb'),
 ('▁и', 'ер'),
 ('▁кес', 'лери'),
 ('▁jибере', 'ди'),
 ('ийг', 'е'),
 ('▁jер', 'леге'),
 ('▁у', 'за'),
 ('▁болса', 'м'),
 ('▁бере', 'йим'),
 ('▁qуча', 'qлаb'),
 ('▁дуния', 'дан'),
 ('▁qой', 'ла'),
 ('▁болу', 'м'),
 ('▁мю', 'лкю'),
 ('▁эл', 'ибиз'),
 ('gа', 'ным'),
 ('▁да', 'рий'),
 ('▁туw', 'ар')]

In [34]:
tokenizer.vocab_size

250054

In [35]:
tokenizer.save_pretrained('old_tokenizer')

('old_tokenizer/tokenizer_config.json',
 'old_tokenizer/special_tokens_map.json',
 'old_tokenizer/sentencepiece.bpe.model',
 'old_tokenizer/added_tokens.json')

In [36]:
import sentencepiece_model_pb2 as model
m = model.ModelProto()
m.ParseFromString(open("old_tokenizer/sentencepiece.bpe.model", "rb").read())

5069051

In [37]:
scores = [p.score for p in m.pieces]
min_score = min(scores)
epsilon = 1e-4

In [38]:
tokenizer('приввет')

{'input_ids': [250004, 37764, 18454, 2], 'attention_mask': [1, 1, 1, 1]}

In [39]:
type(m.pieces[37764-1].piece)

str

In [40]:
for i, pair in enumerate(extra_vocab):
    new_token = model.ModelProto().SentencePiece()
    new_token.piece = ''.join(pair)
    new_token.score = min_score - epsilon * (i+1)
    m.pieces.append(new_token)

In [41]:
with open("old_tokenizer/sentencepiece.bpe.model", 'wb') as f:
    f.write(m.SerializeToString())

In [42]:
new_tokenizer = MBart50Tokenizer.from_pretrained(
    "old_tokenizer/sentencepiece.bpe.model", 
    additional_special_tokens = tokenizer.additional_special_tokens
)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling MBart50Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [43]:
new_tokenizer.vocab_size

252291

In [44]:
from transformers.models.mbart50.tokenization_mbart50 import FAIRSEQ_LANGUAGE_CODES

In [45]:
self = new_tokenizer

In [46]:
self.lang_code_to_id = {
    code: self.sp_model_size + i + self.fairseq_offset for i, code in enumerate(FAIRSEQ_LANGUAGE_CODES + ['qm_XX'])
}
self.id_to_lang_code = {v: k for k, v in self.lang_code_to_id.items()}
self.fairseq_tokens_to_ids["<mask>"] = len(self.sp_model) + len(self.lang_code_to_id) + self.fairseq_offset

self.fairseq_tokens_to_ids.update(self.lang_code_to_id)
self.fairseq_ids_to_tokens = {v: k for k, v in self.fairseq_tokens_to_ids.items()}



In [47]:
new_tokenizer.additional_special_tokens.append('qm_XX')

In [48]:
new_tokenizer.vocab_size

252292

In [49]:
len(new_tokenizer)

252292

In [50]:
text = random.choice(all_sentences)
print(text)

Сора бу халалjюрек Мурат, атасыны байлыgын, асырамай, jарлы халqgа аqырын-аqырын юлеше, кеси jарлы болуb qалады.


In [51]:
print(tokenizer.tokenize(text))
print(new_tokenizer.tokenize(text))

['▁Со', 'ра', '▁бу', '▁ха', 'лал', 'j', 'ю', 'рек', '▁Мур', 'ат', ',', '▁атасы', 'ны', '▁бай', 'лы', 'g', 'ын', ',', '▁асыра', 'май', ',', '▁j', 'ар', 'лы', '▁хал', 'q', 'g', 'а', '▁а', 'q', 'ы', 'рын', '-', 'а', 'q', 'ы', 'рын', '▁', 'ю', 'ле', 'ше', ',', '▁кес', 'и', '▁j', 'ар', 'лы', '▁болу', 'b', '▁q', 'а', 'лады', '.']
['▁Сора', '▁бу', '▁халал', 'j', 'ю', 'рек', '▁Мурат', ',', '▁атасыны', '▁бай', 'лыgын', ',', '▁асыра', 'май', ',', '▁jарлы', '▁халqgа', '▁аqы', 'рын', '-', 'а', 'q', 'ы', 'рын', '▁', 'ю', 'ле', 'ше', ',', '▁кеси', '▁jарлы', '▁болуb', '▁qалады', '.']


In [52]:
len(all_sentences)

27235

In [53]:
random.seed(1)
sample = random.sample(list(all_sentences), 10000)

In [54]:
sample[:10]

['Гапалаwла эки элле – бир болду, гапалаwну jатар jери кёр болду, анасындан jигит туwgан гапалаw!',
 '– Сиз аны ючюн jарсымаgыз, ма бу юйюдю, – деb, qонаqланы сыйлаb, артха ашырды.',
 'Атаwул',
 'Ким бла сёлеширге сюесиз?',
 'Бир кюн jатаgында jуqлаb турgанлай, jуqусундан уяныb олтурgанлай, бир ариw qанатлы юсюнден учхан эди, qызны jюрегин qорqуw алgан эди.',
 '– Qазандаgы кючлю хан чабарыq тюлдю, чораны wa qайgысы узаqда юйюдю.',
 'Аманлыqны jолун бирден алgансыз, элигизге qайытмайын qалgансыз.',
 'Ол ханны, кёb болмай, qызы тас болgанды.',
 'Тиширыw.',
 'Иги хазырланыb, qайтыb, эмеген бла сермеширге келгенди.']

In [55]:
pd.DataFrame({
    'old': [len(tokenizer.tokenize(text)) for text in sample], 
    'new': [len(new_tokenizer.tokenize(text)) for text in sample]
}).describe()

,old,new
count,10000.000000,10000.000000
mean,32.009300,22.614300
std,21.447424,15.492112
min,1.000000,1.000000
25%,16.000000,11.000000
50%,31.000000,21.000000
75%,43.000000,30.000000
max,303.000000,214.000000


In [ ]:
# new_tokenizer.convert_tokens_to_ids('▁Сталинти')

3

In [56]:
new_tot_len, tot_len = 0, 0
for text in sample:
    for tok in new_tokenizer.tokenize(text):
        s = len(tok)
        tot_len += s
        if new_tokenizer.convert_tokens_to_ids(tok) > len(tokenizer.sp_model):
            new_tot_len += s
print(new_tot_len / tot_len)

0.46904326717121236


In [57]:
new_tokenizer('Бу Эресейди')

{'input_ids': [252241, 10962, 27576, 103, 64089, 1752, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [58]:
len(new_tokenizer)

252292

In [59]:
len(m.pieces)

252237

In [ ]:
# model.resize_token_embeddings(len(tokenizer))
# todo: move language id embeddings by the distance. 

In [60]:
new_tokenizer.additional_special_tokens.append('qm_XX')

# Calculating tokens alignment

In [61]:
all_sent = pd.read_csv('/Til/MyDrive/Colab Notebooks/Til/Data/Bary_bir_to_model.csv')
all_pairs = list()

In [62]:
for i in range(len(all_sent)):
    all_pairs.append(all_sent.iloc[i].tolist())

In [63]:
all_pairs[:10]

[['Теnиз тузлу некди?', 'Почему море солёное?'],
 ['Бир jашчыq анасы бла тиnизге jуwунурgа барgанды. Jашчыq qуwаныb, суwgа кёмюлсе, аwузуна суw киргенди, кеси wa тузлу суw. Jунчуgан да этиb, бир уртлам суwну да jутуb, ол jаgаgа чыqgанды. Суwну алай тузлу нек болgанын аnыламай:',
  'Один мальчик со своей мамой на море купаться поехал.  Мальчик был рад, в воду вошёл, вода в рот попала, солёная по вкусу. Мальчик растерялся, сделал глоток воды и проглотил, и затем на берег выбежал. Вода почему стала солёной он не понял:'],
 ['- Некди теnиз былай тузлу? – деb сейирсинnенди.',
  '- Почему вода такая солёная? – он поинтересовался.'],
 ['Анасы jашына теnизни хапарын айтханды:',
  'Мама мальчика о море начала говорить:'],
 ['- Эртде-эртде бир бай qарнаш бла бир jарлы qарнаш jашаgандыла. Jарлы qарнаш байgа «менnе муну бла бир тюйчюк бер» деb, табаqны алыb, тилей барgанды. Бай qарнаш бермегенди.',
  '- Давным-давно жили один богатый брат и один бедный брат. Бедный брат богатому: «мне пшёнку дай» 

In [64]:
random.choice(all_pairs)

['Ол да qалай этерге билмей турgанлай, эмегенлени алларында келгени уруgа ташайды.',
 'Находясь там, он не знал, что делать и тот, который шел впереди эмегенов, провалился в эту же яму.']

In [65]:
from collections import defaultdict, Counter
token_priors = Counter()
token_to_others = defaultdict(Counter)

In [66]:
for qm, ru in tqdm(all_pairs):
    qm_toks = new_tokenizer.convert_tokens_to_ids(new_tokenizer.tokenize(qm))
    ru_toks = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ru))
    token_priors.update(ru_toks)
    for qm_tok in qm_toks:
        token_to_others[qm_tok].update(ru_toks)

  0%|          | 0/27235 [00:00<?, ?it/s]

In [67]:
qm, ru = random.choice(all_pairs)
qm, ru

('Qулларыnы санамайса адамgа, туqумуnа салдыn сен кетмез тамgа.',
 'Своих кулов ты за людей не считаешь, на своей фамилии оставляешь несмываемое пятно.')

In [68]:
print(new_tokenizer.convert_tokens_to_ids(new_tokenizer.tokenize(qm)))

[2396, 9080, 251643, 32604, 252047, 250955, 4, 251135, 105, 19, 59, 174345, 19, 41787, 44512, 888, 1316, 3926, 177, 59, 5]


In [69]:
qm_tok = 251606

In [70]:
import numpy as np

def get_ru_toks(qm_tok):
    ru_toks = []
    ru_weights = []
    for t, w in token_to_others[qm_tok].items():
        ru_toks.append(t)
        ru_weights.append(w**2 / token_priors[t])
    ru_weights = np.array(ru_weights)
    ru_weights = ru_weights / (sum(ru_weights) + 1e-4)
    return ru_weights, ru_toks

In [71]:
sorted(zip(*get_ru_toks(qm_tok)), reverse=True)[:5]

[(0.21708119136906784, 100608),
 (0.04495764318294305, 221867),
 (0.04495764318294305, 160372),
 (0.04495764318294305, 111712),
 (0.04495764318294305, 57570)]

In [72]:
tokenizer.convert_ids_to_tokens([62994, 17921, 117229, 188048, 25982])

['▁первую', '▁справ', '▁карты', 'ведению', '▁ат']

# Updating the model embeddings

In [73]:
len(new_tokenizer)

252292

In [74]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [139]:
# from transformers import M2M100ForConditionalGeneration, MBart50Tokenizer
# model = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_418M')

In [75]:
model.model.shared

Embedding(250054, 1024, padding_idx=1)

In [76]:
model.resize_token_embeddings(len(new_tokenizer))

Embedding(252292, 1024)

Move added tokens to the end

In [77]:
n_extra = len(extra_vocab)
print(n_extra)
old_vocab_size = len(tokenizer.sp_model) + 1

2237


In [78]:
for old_token_id in range(old_vocab_size, len(tokenizer)):
    old_token = tokenizer.convert_ids_to_tokens(old_token_id)
    new_token_id = new_tokenizer.convert_tokens_to_ids(old_token)
    
    print(old_token_id, old_token, new_token_id)
    # model.model.shared.weight.data[i + n_extra] = model.model.shared.weight.data[i]
    model.model.shared.weight.data[new_token_id] = model.model.shared.weight.data[old_token_id]

250001 ar_AR 252238
250002 cs_CZ 252239
250003 de_DE 252240
250004 en_XX 252241
250005 es_XX 252242
250006 et_EE 252243
250007 fi_FI 252244
250008 fr_XX 252245
250009 gu_IN 252246
250010 hi_IN 252247
250011 it_IT 252248
250012 ja_XX 252249
250013 kk_KZ 252250
250014 ko_KR 252251
250015 lt_LT 252252
250016 lv_LV 252253
250017 my_MM 252254
250018 ne_NP 252255
250019 nl_XX 252256
250020 ro_RO 252257
250021 ru_RU 252258
250022 si_LK 252259
250023 tr_TR 252260
250024 vi_VN 252261
250025 zh_CN 252262
250026 af_ZA 252263
250027 az_AZ 252264
250028 bn_IN 252265
250029 fa_IR 252266
250030 he_IL 252267
250031 hr_HR 252268
250032 id_ID 252269
250033 ka_GE 252270
250034 km_KH 252271
250035 mk_MK 252272
250036 ml_IN 252273
250037 mn_MN 252274
250038 mr_IN 252275
250039 pl_PL 252276
250040 ps_AF 252277
250041 pt_XX 252278
250042 sv_SE 252279
250043 sw_KE 252280
250044 ta_IN 252281
250045 te_IN 252282
250046 th_TH 252283
250047 tl_XX 252284
250048 uk_UA 252285
250049 ur_PK 252286
250050 xh_ZA 252287


Because we have added one more language, its id must be computed separately, e.g. as an average of related languages.

In [79]:
model.model.shared.weight.data[new_tokenizer.convert_tokens_to_ids('qm_XX')] = (
    model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('kk_KZ')] * 0.6
    + model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('az_AZ')] * 0.2
    + model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('ru_RU')] * 0.1
)

Compute embeddings for newly added tokens

In [80]:
for i in trange(n_extra):
    qm_tok = i + old_vocab_size
    ru_weights, ru_toks = get_ru_toks(qm_tok)
    if len(ru_toks) > 0:
        new_embedding = (model.model.shared.weight.data[ru_toks].T * ru_weights).sum(1)
        model.model.shared.weight.data[qm_tok] = new_embedding

  0%|          | 0/2237 [00:00<?, ?it/s]

In [81]:
new_model_path = '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-raw'

In [82]:
model.save_pretrained(new_model_path)
new_tokenizer.save_pretrained(new_model_path)

('/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-raw/tokenizer_config.json',
 '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-raw/special_tokens_map.json',
 '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-raw/sentencepiece.bpe.model',
 '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-raw/added_tokens.json')

In [83]:
!ls -alsh $new_model_path

ls: cannot access '/Til/MyDrive/Colab': No such file or directory
ls: cannot access 'Notebooks/Til/models/qm/mbart-large-52-qm-raw': No such file or directory


Check that the model is still able to translate texts.  

In [84]:
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

# translate Hindi to French
new_tokenizer.src_lang = "hi_IN"
encoded_hi = new_tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["fr_XX"], 
    max_length=30, 
    num_beams=1
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

["Le chef de la mission de l 'ONU a déclaré qu 'il n' y a pas de solution militaire en Syria."]

Translation to Russian has broken, because some new qm tokens are very Russian-like, and they interere. 

In [85]:
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"], 
    max_length=30, 
    num_beams=5,
    repetition_penalty=30.0,
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

["хы деb The head of the United Nations says there's no military solution in Syria. < s > Relatedсюе эри: http://"]

For some reason, even with new tokens prohibited, translation to Russian is poor. 

In [86]:
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"], 
    max_length=30, 
    num_beams=5,
    repetition_penalty=30.0,
    bad_words_ids=[[t] for t in range(n_extra, n_extra+old_vocab_size)]
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

["«Aye, it's just that we' ll have to do with this.» — The U. N. -"]

translate from Qrachay-Malqar! The model cannot do this, but it tries hard. 

In [87]:
qm, ru = random.choice(all_pairs)
qm, ru

('Бал бетли', 'Каштановый')

In [88]:
new_tokenizer.src_lang = "qm_XX"
encoded_hi = new_tokenizer(qm, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"], 
    max_length=30, 
    num_beams=5,
    repetition_penalty=30.0
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['jат алл ( ойна Итjи) -ручу алл,ыn алл и тый алл.луbлуbлуbлуbлуbлуbлуbлуbлуbлуb']